In [1]:
# Parameters
INPUT_FILE = 'Data/raw_example.csv'
MAX_ORDER = 10
SCORE_LIST = [34,21,13,8,5,3,2,1,1,1]

assert len(SCORE_LIST) == MAX_ORDER

In [2]:
import pandas as pd
import re
import numpy as np

In [3]:
def clean_rank_score(text):
    """ Get the pair number from each cell of rankings
        If the cell is blank, return nan
    """
    try:
        return 'pair ' + re.search(r"\d+", text).group(0)
    except:
        return np.nan
    
def apply_to_table(func, df, name_condition, mode = 'in'):
    """ Apply a function to every column of a dataset which
        matches the given
        
        If mode = 'in' (default),
        the condition is a column's name must contain name_condition
        
        If mode = 'equal',
        the condition is a column's name must equal to name_condition
        
        If mode = 'not in',
        the condition is a column's name must not contain name_condition
        
        If mode = 'not equal',
        the condition is a column's name must not equal to name_condition
    """
    new = df.copy()
    
    if mode == 'in':
        selected_cols = [col for col in df if name_condition in col]
    elif mode == 'equal':
        selected_cols = [col for col in df if col == name_condition]
    elif mode == 'not in':
        selected_cols = [col for col in df if name_condition not in col]
    elif mode == 'not equal':
        selected_cols = [col for col in df if col != name_condition]
    else:
        return "Error: mode is not defined"
    
    for col in new.columns:
        if col in selected_cols:
            new[col] = df[col].apply(func)
    return new

In [4]:
# The first three rows (exclude header) are trial rows (we won't use it)
data = pd.read_csv(INPUT_FILE)

In [5]:
new_col_names = ['timestamp', 'email', 'tel', 'ID', 'name']
new_col_names.extend([''.join(['rank', str(i + 1)]) for i in range(MAX_ORDER)])
data.columns = new_col_names

# Remove duplicates (keep last)
data.drop_duplicates(subset = 'ID', keep = 'last', inplace = True)

# Edit the one of which ID is wrong (60317499221 -> 6031749221)
data.replace({60317499221: 6031749221}, inplace = True)

# Set index
data.set_index('ID', inplace = True)

#data.head(3)

In [6]:
# Drop unused columns
data.drop(['timestamp', 'email', 'tel', 'name'], axis = 1, inplace = True)
#data.head(3)

In [7]:
ranks = data[[col for col in data if 'rank' in col]]
#ranks.head(3)

In [8]:
rank_cleaned = apply_to_table(clean_rank_score, ranks, 'rank', 'in')
#rank_cleaned

## Flag Check: Select Duplicate Pairs

If a candidate select the same pair in a different rank, we will use only the lower rank and will replace the higher rank with some random pair that is not selected much.

In [9]:
def dup_flag(ranks):
    unique_list = []
    dup_list = []
    for pair in ranks:
        if pair not in unique_list:
            unique_list.append(pair)
        else:
            dup_list.append(pair)
    return '-'.join(dup_list)

In [10]:
rank_cleaned['dup_flag'] = rank_cleaned.apply(dup_flag, axis = 0)
rank_cleaned.head(3)

,rank1,rank2,rank3,rank4,rank5,rank6,rank7,rank8,rank9,rank10,dup_flag
ID,,,,,,,,,,,
6230124521,pair 17,pair 2,pair 1,pair 5,pair 15,pair 18,pair 10,pair 12,pair 8,pair 6,NaN
6031843521,pair 16,pair 8,pair 3,pair 17,pair 7,pair 10,pair 13,pair 6,pair 12,pair 18,NaN
6030274221,pair 3,pair 15,pair 12,pair 18,pair 8,pair 2,pair 5,pair 17,pair 6,pair 10,NaN


In [11]:
# Check if there is any duplicates in our data
rank_cleaned['dup_flag'].value_counts()

Series([], Name: dup_flag, dtype: int64)

If there is no duplicate, move on to the next step.

# Create a New Table

In [12]:
col_list = [' '.join(['pair', str(i)]) for i in range(1, 19)]

# Drop duplicate flag from rank_cleaned
rank_cleaned.drop(['dup_flag'], axis = 1, inplace = True)

scores = pd.DataFrame(index=rank_cleaned.index, columns=col_list)

In [13]:
for candidate_id in rank_cleaned.index:
    row = rank_cleaned.loc[candidate_id]
    for rank in row.index:
        # Remove the word "rank"
        rank_number = rank[4:]
        
        scores.loc[candidate_id, row.loc[rank]] = rank_number
        
# Reorder columns
pair_order = [' '.join(['pair', str(i)]) for i in range(1, 19)]
scores = scores[pair_order]

In [14]:
#scores

In [22]:
# Save
scores.to_csv('pair_selection.csv')

In [16]:
data.rank1.value_counts()

Pair 17 : พี่ใหม่ ปริมล กาญจนจารี, พี่วิน วิธวินท์ อิทธิภาณุวัต                 24
Pair 3 : พี่โบ้ พิเชษฐ สิทธิอำนวย, พี่เหมย ศิรินทร์ เจ้าพิทักษ์วงศ์             12
Pair 8 : พี่ประทิต ประทิต โลหะธีรภาพ, พี่ตั้ม สันติ ศรีวิชาญกุล                 11
Pair 1 : พี่เล้ง ศิริวัฒน์ วงศ์จารุกร, พี่โทนี่ โทนี่ คันธาภัสระ                 9
Pair 15 : พี่อ้อ รัชดา ด่านพงษ์เจริญ, พี่โอห์ม อัฐพงศ์ รัตนวีระชานน              8
Pair 11 : พี่ไมเคิล อธิวรรธน์ วงศ์ไวศยวรรณ, พี่ธัช กิตติธัช อภิศักดิ์ศิริกุล     7
Pair 2 : พี่ก้อง พณชิต กิตติปัญญางาม, พี่ต้น วงศกร ชัยวนนท์                      7
Pair 18 : พี่ต้อง ดร. อโณทัย รัตนกุล, พี่ต้า ดร. วิโรจน์ จิรพัฒนกุล              6
Pair 5 : พี่เก่ง ดร.ธิติ วัชรสินธพชัย, พี่วี วีรวัฒน์ คิรินทร์รัตนะ              6
Pair 13 : พี่โน อโณทัย อดุลพันธุ์, พี่หวัง กิตติศักดิ์ ปัญญาจิรกุล               3
Pair 12 : พี่เส เสนธิป ศรีไพพรรณ, พี่แจ็ค ชวิน ศุภวงศ์                           3
Pair 9 : พี่รุจ ดร. รุจ ณ สงขลา, พี่ต้อง สโรชินี ปวีณวัฒน์                       3
Pair

## Add Tier

In [17]:
app_score = pd.read_excel('Data/application_score.xlsx', sheet_name='final', index_col='ID')

In [18]:
score_with_tier = scores.copy()

In [19]:
score_with_tier['tier'] = app_score['Status']
score_with_tier.head()

,pair 1,pair 2,pair 3,pair 4,pair 5,pair 6,pair 7,pair 8,pair 9,pair 10,pair 11,pair 12,pair 13,pair 14,pair 15,pair 16,pair 17,pair 18,tier
ID,,,,,,,,,,,,,,,,,,,
6230124521,3,2,NaN,NaN,4,10,NaN,9,NaN,7,NaN,8,NaN,NaN,5,NaN,1,6,4
6031843521,NaN,NaN,3,NaN,NaN,8,5,2,NaN,6,NaN,9,7,NaN,NaN,1,4,10,2
6030274221,NaN,6,1,NaN,7,9,NaN,5,NaN,10,NaN,3,NaN,NaN,2,NaN,8,4,4
6031779021,4,NaN,2,NaN,NaN,NaN,NaN,1,5,6,NaN,3,NaN,NaN,7,8,10,9,4
6030826721,10,1,4,NaN,NaN,NaN,NaN,5,NaN,NaN,9,6,2,NaN,8,NaN,3,7,1


In [23]:
score_with_tier.to_csv('pair_selection_with_tier.csv')

In [25]:
score_with_tier.tier.value_counts()

4    41
2    29
3    27
1    11
Name: tier, dtype: int64